#### Option 1: sentiment on English texts

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch

sentiment = SentimentIntensityAnalyzer()

text_1 = "By rejecting God, you are rejecting goodness, kindness, and love itself."
text_2 = "We, the people, are going to work together to achieve justice, unity, and integrity."
text_3 = "Ms. Baker assigned me a lot of homework because she’s a witch!"

sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)
sent_3 = sentiment.polarity_scores(text_3)

print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)
print("Sentiment of text 3:", sent_3)

#### Option 2: sentiment on Romanian texts

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "dumitrescustefan/bert-base-romanian-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    sentiment = torch.argmax(logits).item()
    labels = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return labels[sentiment]


text = "Îmi place foarte mult acest film!"
print(analyze_sentiment(text))

#### Option 3: sentiment on Romanian texts


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "dumitrescustefan/bert-base-romanian-uncased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    sentiment = torch.argmax(logits).item()
    labels = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return labels[sentiment]


text = "Îmi place foarte mult acest film!"
print(analyze_sentiment(text))

#### Option 4: sentiment on Romanian texts
 
https://huggingface.co/readerbench/ro-sentiment
Output labels:

LABEL_0 = Negative Sentiment
LABEL_1 = Positive Sentiment

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "readerbench/ro-sentiment"

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


def sentiment(text):
    """Assess the sentiment of the input text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits.squeeze().cpu()
    return logits.tolist()


# Example usage
score = sentiment("Îmi place foarte mult acest film!")
print(f"Sentiment score: {score}")

### Final version

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df = pd.read_csv("../data/all/combined_lfud_huggingface_binary.csv")

analyzer = SentimentIntensityAnalyzer()


def get_sentiment(text):
    if pd.isna(text):
        print("There is a nan value")
    score = analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"


# Apply sentiment analysis to 'source_article'
df['sentiment'] = df['source_article'].apply(get_sentiment)

In [ ]:
df.to_csv("../data/all/combined_lfud_huggingface_binary_sent.csv", index=False)
print(df.head())

In [ ]:
data = pd.read_csv("../data/all/combined_lfud_huggingface_binary_sent.csv")

In [ ]:
data.value_counts(data["sentiment"])